# DATAFRAME PROJECT

___

## Use the Netflix_2011_2016.csv file to Answer and complete the commented tasks below!

### 1. Start a simple Spark Session

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7f662d75


### 2. Load the Netflix Stock CSV File, have Spark infer the data types.

In [2]:
val df = spark.read.options(Map(("header","true"),("inferSchema","true"))).csv("Netflix_2011_2016.csv")

df: org.apache.spark.sql.DataFrame = [Date: timestamp, Open: double ... 5 more fields]


### 3. What are the column names?

In [4]:
df.columns

res1: Array[String] = Array(Date, Open, High, Low, Close, Volume, Adj Close)


### 4. What does the Schema look like?

In [5]:
df.printSchema

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### 5. Print out the first 5 columns.

In [6]:
for (line <- df.head(5)){
    println(line)
}

[2011-10-24 00:00:00.0,119.100002,120.28000300000001,115.100004,118.839996,120460200,16.977142]
[2011-10-25 00:00:00.0,74.899999,79.390001,74.249997,77.370002,315541800,11.052857000000001]
[2011-10-26 00:00:00.0,78.73,81.420001,75.399997,79.400002,148733900,11.342857]
[2011-10-27 00:00:00.0,82.179998,82.71999699999999,79.249998,80.86000200000001,71190000,11.551428999999999]
[2011-10-28 00:00:00.0,80.280002,84.660002,79.599999,84.14000300000001,57769600,12.02]


### 6. Use describe() to learn about the DataFrame.

In [7]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+------------------+
|summary|              Open|              High|               Low|             Close|              Volume|         Adj Close|
+-------+------------------+------------------+------------------+------------------+--------------------+------------------+
|  count|              1259|              1259|              1259|              1259|                1259|              1259|
|   mean|230.39351086656092|233.97320872915006|226.80127876251044|  230.522453845909|2.5634836060365368E7|55.610540036536875|
| stddev|164.37456353264244| 165.9705082667129| 162.6506358235739|164.40918905512854| 2.306312683388607E7|35.186669331525486|
|    min|         53.990001|         55.480001|             52.81|              53.8|             3531300|          7.685714|
|    max|        708.900017|        716.159996|        697.569984|        707.610001|           315541800|        130.

### 7. Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [8]:
val df2 = df.withColumn("HV_Ratio",df("High")/df("Volume"))

df2: org.apache.spark.sql.DataFrame = [Date: timestamp, Open: double ... 6 more fields]


In [12]:
df2.select("HV_Ratio").show(5,false)

+---------------------+
|HV_Ratio             |
+---------------------+
|9.985040951285156E-7 |
|2.515989989281927E-7 |
|5.474206014903126E-7 |
|1.1619609074308188E-6|
|1.4654766867002715E-6|
+---------------------+
only showing top 5 rows



### 8. What day had the Peak High in Price?

In [27]:
df.select("Date","High").orderBy($"High".desc).show(1)

+-------------------+----------+
|               Date|      High|
+-------------------+----------+
|2015-07-13 00:00:00|716.159996|
+-------------------+----------+
only showing top 1 row



Or

In [49]:
val highPrice = df.select(max("High")).collect().map(x => x.getDouble(0))

highPrice: Array[Double] = Array(716.159996)


In [50]:
highPrice(0)

res37: Double = 716.159996


In [57]:
df.select("Date","High").filter("High = " + highPrice(0)).show()

+-------------------+----------+
|               Date|      High|
+-------------------+----------+
|2015-07-13 00:00:00|716.159996|
+-------------------+----------+



Or

In [58]:
df.select("Date","High").filter($"High" === highPrice(0)).show()

+-------------------+----------+
|               Date|      High|
+-------------------+----------+
|2015-07-13 00:00:00|716.159996|
+-------------------+----------+



### 9. What is the mean of the Close column?

In [59]:
df.select(mean("Close")).show()

+----------------+
|      avg(Close)|
+----------------+
|230.522453845909|
+----------------+



### 10. What is the max and min of the Volume column?

In [60]:
df.select(max("Volume")).show()

+-----------+
|max(Volume)|
+-----------+
|  315541800|
+-----------+



In [61]:
df.select(min("Volume")).show()

+-----------+
|min(Volume)|
+-----------+
|    3531300|
+-----------+



### 11. For Scala/Spark $ Syntax

In [62]:
import spark.implicits._

import spark.implicits._


### 12. How many days was the Close lower than $ 600?

In [63]:
df.filter($"Close" < 600).count()

res49: Long = 1218


Or

In [64]:
df.filter("Close < 600").count()

res50: Long = 1218


### 13. What percentage of the time was the High greater than $500 ?

In [69]:
(df.filter($"High" > 500).count())*100.0/df.count()

res55: Double = 4.924543288324067


### 14. What is the Pearson correlation between High and Volume?

In [71]:
df.select(corr("High","Volume")).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.20960233287942157|
+--------------------+



### 15. What is the max High per year?

In [72]:
df.show(3)

+-------------------+----------+------------------+----------+----------+---------+------------------+
|               Date|      Open|              High|       Low|     Close|   Volume|         Adj Close|
+-------------------+----------+------------------+----------+----------+---------+------------------+
|2011-10-24 00:00:00|119.100002|120.28000300000001|115.100004|118.839996|120460200|         16.977142|
|2011-10-25 00:00:00| 74.899999|         79.390001| 74.249997| 77.370002|315541800|11.052857000000001|
|2011-10-26 00:00:00|     78.73|         81.420001| 75.399997| 79.400002|148733900|         11.342857|
+-------------------+----------+------------------+----------+----------+---------+------------------+
only showing top 3 rows



In [74]:
var df3 = df.withColumn("Year",year($"Date"))

df3: org.apache.spark.sql.DataFrame = [Date: timestamp, Open: double ... 6 more fields]


In [76]:
df3.show(3)

+-------------------+----------+------------------+----------+----------+---------+------------------+----+
|               Date|      Open|              High|       Low|     Close|   Volume|         Adj Close|Year|
+-------------------+----------+------------------+----------+----------+---------+------------------+----+
|2011-10-24 00:00:00|119.100002|120.28000300000001|115.100004|118.839996|120460200|         16.977142|2011|
|2011-10-25 00:00:00| 74.899999|         79.390001| 74.249997| 77.370002|315541800|11.052857000000001|2011|
|2011-10-26 00:00:00|     78.73|         81.420001| 75.399997| 79.400002|148733900|         11.342857|2011|
+-------------------+----------+------------------+----------+----------+---------+------------------+----+
only showing top 3 rows



In [79]:
df3.groupBy("Year").max("High").orderBy("Year").show()

+----+------------------+
|Year|         max(High)|
+----+------------------+
|2011|120.28000300000001|
|2012|        133.429996|
|2013|        389.159988|
|2014|        489.290024|
|2015|        716.159996|
|2016|129.28999299999998|
+----+------------------+



### 16. What is the average Close for each Calender Month?

In [83]:
df.groupBy(month($"Date")).avg("Close").orderBy(month($"Date")).show()

+-----------+------------------+
|month(Date)|        avg(Close)|
+-----------+------------------+
|          1|212.22613874257422|
|          2| 254.1954634020619|
|          3| 249.5825228971963|
|          4|246.97514271428562|
|          5|264.37037614150944|
|          6| 295.1597153490566|
|          7|243.64747528037387|
|          8|195.25599892727263|
|          9|206.09598121568627|
|         10|205.93297300900903|
|         11| 194.3172275445545|
|         12| 199.3700942358491|
+-----------+------------------+



### Closing Spark Session

In [84]:
spark.stop()

## Thank You!